In [ ]:
import os
import glob
import pandas as pd
import numpy as np
from copy import deepcopy

In [ ]:
run_directory = r"\\tore-infs01\Models\VaughanTMP\2051_ForecastRuns"

run_folder = r"2051_Preferred_new_20211209_1"
run_path = os.path.join(run_directory,run_folder)

input_pth = r"C:\Users\pechen\Desktop\VTP\SkM"
zone_map = pd.read_csv(os.path.join(input_pth,"VTP_ZoneMapping.csv"))
zone_map_min = zone_map

In [3]:
microsim_location = r"Microsim Results"
microsim_path = os.path.join(run_path, microsim_location)

persons_df = pd.read_csv(os.path.join(microsim_path,"persons.csv"))
trips_df = pd.read_csv(os.path.join(microsim_path,"trips.csv"))
tripmode_df = pd.read_csv(os.path.join(microsim_path,"trip_modes.csv"))
trips_gta = trips_df
[(trips_df.o_zone < 6000) & (trips_df.d_zone < 6000)]

trips_gta = trips_gta.merge(persons_df,on=["household_id","person_id"])
trips_gta = trips_gta[trips_gta.age >=11]

trips_gta_time = tripmode_df.merge(trips_gta,how = "right", on = ['household_id', 'person_id', 'trip_id'])

trips_gta_time["weight"] = trips_gta_time["weight"]/10
trips_gta_time["weight_z"] = trips_gta_time["weight"]*trips_gta_time["weight_x"]

ms_daily_GTA = pd.pivot_table(trips_gta_time,columns = "mode", values = "weight_z", aggfunc = sum)

In [4]:
trips_VTP = trips_df
trips_VTP  = trips_VTP.merge(persons_df,on=["household_id","person_id"])
trips_VTP = trips_VTP[trips_VTP.age >=11]
trips_VTP = trips_VTP.merge(zone_map_min,left_on = "o_zone", right_on ="Zone")
trips_VTP = trips_VTP.merge(zone_map_min,left_on = "d_zone", right_on ="Zone")

trips_VTP_time = tripmode_df.merge(trips_VTP,how = "right", on = ['household_id', 'person_id', 'trip_id'])
trips_VTP_time["weight"] = trips_VTP_time["weight"]/10
trips_VTP_time["weight_z"] = trips_VTP_time["weight"]*trips_VTP_time["weight_x"]
trips_VTP_am = trips_VTP_time[(trips_VTP_time.o_depart >= 360) & (trips_VTP_time.o_depart < 540)]
trips_VTP_pm = trips_VTP_time[(trips_VTP_time.o_depart >= 540) & (trips_VTP_time.o_depart < 900)]

In [7]:
am_modeshare = pd.pivot_table(trips_VTP_am,index = ["o_zone","d_zone"],columns = "mode", values = "weight_z", aggfunc = sum,margins = True).fillna(0)
pm_modeshare = pd.pivot_table(trips_VTP_pm,index = ["o_zone","d_zone"],columns = "mode", values = "weight_z", aggfunc = sum,margins = True).fillna(0)
daily_modeshare = pd.pivot_table(trips_gta_time,index = ["o_zone","d_zone"],columns = "mode", values = "weight_z", aggfunc = sum,margins = True).fillna(0)


In [ ]:
# am_active = am_modeshare.copy(deep=True)
# am_active['active'] = am_active['Bicycle'] + am_active['Walk']
# am_active.drop(am_active.columns.difference(['active','o_zone', 'd_zone']), 1, inplace=True)
# am_active.drop(am_active.tail(1).index,inplace=True)

# pm_active = pm_modeshare.copy(deep=True)
# pm_active['active'] = pm_active['Bicycle'] + pm_active['Walk']
# pm_active.drop(pm_active.columns.difference(['active','o_zone', 'd_zone']), 1, inplace=True)
# pm_active.drop(pm_active.tail(1).index,inplace=True)

In [ ]:
am_auto = am_modeshare.copy(deep=True)
am_auto.drop(am_auto.columns.difference(['Auto','o_zone', 'd_zone']), 1, inplace=True)
am_auto.drop(am_auto.tail(1).index,inplace=True)

pm_auto = pm_modeshare.copy(deep=True)
pm_auto.drop(pm_auto.columns.difference(['Auto','o_zone', 'd_zone']), 1, inplace=True)
pm_auto.drop(pm_auto.tail(1).index,inplace=True)

daily_auto = daily_modeshare.copy(deep=True)
daily_auto.drop(daily_auto.columns.difference(['Auto','o_zone', 'd_zone']), 1, inplace=True)
daily_auto.drop(daily_auto.tail(1).index,inplace=True)

In [ ]:
am_transit = am_modeshare.copy(deep=True)
am_transit['transit'] = am_transit['PAT']+am_transit['DAT']+am_transit['PET']+am_transit['WAT']
am_transit.drop(am_transit.columns.difference(['transit','o_zone', 'd_zone']), 1, inplace=True)
am_transit.drop(am_transit.tail(1).index,inplace=True)

pm_transit = pm_modeshare.copy(deep=True)
pm_transit['transit'] = pm_transit['PAT']+pm_transit['DAT']+pm_transit['PET']+pm_transit['WAT']
pm_transit.drop(pm_transit.columns.difference(['transit','o_zone', 'd_zone']), 1, inplace=True)
pm_transit.drop(pm_transit.tail(1).index,inplace=True)

daily_transit = daily_modeshare.copy(deep=True)
daily_transit['transit'] = daily_transit['PAT']+daily_transit['DAT']+daily_transit['PET']+daily_transit['WAT']
daily_transit.drop(daily_transit.columns.difference(['transit','o_zone', 'd_zone']), 1, inplace=True)
daily_transit.drop(daily_transit.tail(1).index,inplace=True)

In [ ]:
# #am_active = am_active.pivot(index='o_zone', columns='d_zone', values='active')
# am_active.to_csv(os.path.join(input_pth,"am_active_demand.csv"))
# #pm_active = pm_active.pivot(index='o_zone', columns='d_zone', values='active')
# pm_active.to_csv(os.path.join(input_pth,"pm_active_demand.csv"))
am_transit.to_csv(os.path.join(input_pth, "am_transit_demand.csv"))
pm_transit.to_csv(os.path.join(input_pth, "pm_transit_demand.csv"))
daily_transit.to_csv(os.path.join(input_pth, "daily_transit_demand.csv"))
am_auto.to_csv(os.path.join(input_pth, "am_auto_demand.csv"))
pm_auto.to_csv(os.path.join(input_pth, "pm_auto_demand.csv"))
daily_auto.to_csv(os.path.join(input_pth, "daily_auto_demand.csv"))



In [ ]:
# am_active = am_active.pivot(index='o_zone', columns='d_zone', values='active').fillna(0)
# full_index = am_active.index.union(megazones_df.index)                                                                                                                                                                 
# df2 = am_active.reindex(labels=full_index, axis=0).reindex(labels=full_index, axis=1).fillna(0.0)                                                                                                               
# df2.to_csv(os.path.join(input_pth, 'am_active_demand.csv'))

# pm_active = pm_active.pivot(index='o_zone', columns='d_zone', values='active').fillna(0)
# full_index = pm_active.index.union(megazones_df.index)                                                                                                                                                                 
# df3 = pm_active.reindex(labels=full_index, axis=0).reindex(labels=full_index, axis=1).fillna(0.0)                                                                                                               
# df3.to_csv(os.path.join(input_pth, 'pm_active_demand.csv'))

In [ ]:
am_transit = am_transit.pivot(index='o_zone', columns='d_zone', values='transit').fillna(0)
# full_index = am_transit.index.union(megazones_df.index)
df2 = am_transit.reindex(labels=full_index, axis=0).reindex(labels=full_index, axis=1).fillna(0.0)
df2.to_csv(os.path.join(input_pth, 'am_transit_demand.csv'))

pm_transit = pm_transit.pivot(index='o_zone', columns='d_zone', values='transit').fillna(0)
# full_index = pm_transit.index.union(megazones_df.index)
df3 = pm_transit.reindex(labels=full_index, axis=0).reindex(labels=full_index, axis=1).fillna(0.0)
df3.to_csv(os.path.join(input_pth, 'pm_transit_demand.csv'))

daily_transit = daily_transit.pivot(index='o_zone', columns='d_zone', values='transit').fillna(0)
# full_index = pm_transit.index.union(megazones_df.index)
df3 = pm_transit.reindex(labels=full_index, axis=0).reindex(labels=full_index, axis=1).fillna(0.0)
df3.to_csv(os.path.join(input_pth, 'pm_transit_demand.csv'))

In [ ]:
am_transit = am_transit.pivot(index='o_zone', columns='d_zone', values='transit').fillna(0)
# full_index = am_transit.index.union(megazones_df.index)
df2 = am_transit.reindex(labels=full_index, axis=0).reindex(labels=full_index, axis=1).fillna(0.0)
df2.to_csv(os.path.join(input_pth, 'am_transit_demand.csv'))

pm_transit = pm_transit.pivot(index='o_zone', columns='d_zone', values='transit').fillna(0)
# full_index = pm_transit.index.union(megazones_df.index)
df3 = pm_transit.reindex(labels=full_index, axis=0).reindex(labels=full_index, axis=1).fillna(0.0)
df3.to_csv(os.path.join(input_pth, 'pm_transit_demand.csv'))

daily_transit = daily_transit.pivot(index='o_zone', columns='d_zone', values='transit').fillna(0)
# full_index = pm_transit.index.union(megazones_df.index)
df3 = pm_transit.reindex(labels=full_index, axis=0).reindex(labels=full_index, axis=1).fillna(0.0)
df3.to_csv(os.path.join(input_pth, 'pm_transit_demand.csv'))

In [ ]:
am_auto = am_auto.pivot(index='o_zone', columns='d_zone', values='Auto').fillna(0)
full_index = am_auto.index.union(megazones_df.index)
df2 = am_auto.reindex(labels=full_index, axis=0).reindex(labels=full_index, axis=1).fillna(0.0)
df2.to_csv(os.path.join(input_pth, 'am_auto_demand.csv'))

pm_auto = pm_auto.pivot(index='o_zone', columns='d_zone', values='Auto').fillna(0)
full_index = pm_auto.index.union(megazones_df.index)
df3 = pm_auto.reindex(labels=full_index, axis=0).reindex(labels=full_index, axis=1).fillna(0.0)
df3.to_csv(os.path.join(input_pth, 'pm_auto_demand.csv'))